In [2]:
from MCQ import flanT5MCQ

generator_args = {
    "max_new_tokens":150,
#"max_length": 256,
"num_beams": 10, #20
"length_penalty":-0.5, #Since the score is the log likelihood of the sequence (i.e. negative), length_penalty > 0.0 promotes longer sequences, while length_penalty < 0.0 encourages shorter sequences.
"no_repeat_ngram_size": 3,
#'force_words_ids':[[58]],#token of `?` -cannot use constrained beam search with grouped beam search, while `diversity_penalty` can be used only with group beam search.
'top_p' :0.955,
#'do_sample':True,
'diversity_penalty':float(10), #note diversity is calculated between groups, the final scores are across all outputs, therfore the results with highest scores may be from one group and the diversity calcultion won't be effective for large groups
'num_beam_groups':10,#20 
"return_dict_in_generate" :True,
'output_scores':True,
"early_stopping": True, 
'num_return_sequences':8
}

answers_generator_args = {
    "max_new_tokens":150,
    #"max_length": 256,
    "num_beams": 8,#10
    "length_penalty":0.2,
    #"length_penalty": 1.5, #Since the score is the log likelihood of the sequence (i.e. negative), length_penalty > 0.0 promotes longer sequences, while length_penalty < 0.0 encourages shorter sequences.
    "no_repeat_ngram_size": 3,
    #'force_words_ids':[tokenizer.encode(['.'])],
    'top_p' :0.97,
    'diversity_penalty':float(8),
    'num_beam_groups':8,#10,
    "return_dict_in_generate" :True,
    'output_scores':True,
    "early_stopping": True,
    'num_return_sequences':5
}
import json
import copy
import warnings
import numpy as np 
import torch
# Disable all warning messages
warnings.filterwarnings("ignore")
import os
from tqdm.auto import tqdm
tqdm.pandas()
name = 'Financial_Markets_Course2_flanT5L'

dir_path = '../outputs/'+name+'/'
if not os.path.isdir(dir_path):
  os.mkdir(dir_path)

with open('/home/ubuntu/Questions_generation/Financial Markets Course 2.json') as f:
  result = json.load(f)
sections = [s['summary']['text'] if 'summary' in s.keys() and 'text' in s['summary'].keys() else None for k,s in result['sections'].items() ]
org_text = [s['original']['text'] if 'original' in s.keys() and 'text' in s['original'].keys() else None  for k,s in result['sections'].items() ]
min_words_in_section=60
sections = list(filter(lambda x: x is not None and len(x.split())>min_words_in_section, sections))
#org_text = list(filter(lambda x: x is not None and len(x.split())>min_words_in_section, org_text))
short_answers_generator_args = copy.deepcopy(answers_generator_args)
short_answers_generator_args["length_penalty"]=-0.6
mcq = flanT5MCQ(generator_args=generator_args,answers_generator_args=answers_generator_args,short_answers_generator_args=short_answers_generator_args)

cpu_model = mcq.model.to('cpu')
del mcq.model
mcq.model = cpu_model
mcq.push_model_to_GPU(mcq.model)

with torch.no_grad():
  questions_df = mcq.compute_questions(sections=sections,org_sections=org_text,sections_ranks=np.ones(len(sections)))
questions_df.to_pickle(dir_path+'GQ.pickle')
print(f"Stage 1:{dir_path+'GQ.pickle'} has been saved")
torch.cuda.empty_cache()
if mcq.COMPUTE_ANSWERS:
  mcq.push_model_to_GPU(mcq.QA.model)
  questions_df = mcq.QA.select_best_answer(questions_df)
  questions_df.to_pickle(dir_path+'QG+QA.pickle')
  print(f"Stage 2:{dir_path+'QG+QA.pickle'} has been saved")
  questions_df = mcq.QG.create_question_MixQG(questions_df)
  questions_df.to_pickle(dir_path+'GQ+QA+GQ.pickle')
  print(f"Stage 3:{dir_path+'GQ+QA+GQ.pickle'} has been saved")
  
  questions_used = 'new_question'
  questions_df['RQUGE'] = questions_df.apply(lambda x: mcq.rquge.scorer(x.text, x[questions_used], x.selected_ans)[0]['pred_score'] ,axis='columns')
  questions_df = questions_df.sort_values('RQUGE',ascending=False).reset_index()
  filter_idx = mcq.filter_questions(questions_df[questions_used].to_list(),
                                 mcq.find_similarity(questions_df[questions_used].to_list()),
                                 similarity_thrs=0.5, n_thrs=20, return_index=True)
  questions_df = questions_df.iloc[filter_idx,:]
with open(f'{dir_path}questions_full.txt', 'w') as f:
    qs_text =[mcq.show_qs(questions_df,i) for i in questions_df.index.values]
    text_outuput=''
    for i,qs in zip(questions_df.index.values,qs_text):
        text_outuput += f"({i}) RQUGE:{round(questions_df['RQUGE'][i],4)}"+'\n'+ qs+"\n\n"
    f.write(text_outuput)
    print(f"Stage 3:{dir_path}questions.txt has been saved")
    
with open(f'{dir_path}questions.txt', 'w') as f:
    text_outuput=''
    for i in np.unique(questions_df.section_n):
      qs_in_sections = questions_df.loc[questions_df.section_n==i,:]
      for q,a in zip(qs_in_sections[questions_used],qs_in_sections.selected_ans):
        text_outuput+=f'Q:{q}\nA:{a}\n'
      text_outuput+= '-'*50 + '\n'
    f.write(text_outuput)
    print(f"Stage 4:{dir_path}questions.txt has been saved")

Token indices sequence length is longer than the specified maximum sequence length for this model (963 > 512). Running this sequence through the model will result in indexing errors


Abbreviation 	 Definition
{}


0it [00:00, ?it/s]

Total text shape is torch.Size([1, 376])
Qs:Why was there a pre-break around 2000?
Qs:Who were some of these institutions that collapsed?
Qs:The speaker is going to talk about what in the context?
Qs:How did this lecture begin?
Qs:When the stock market collapsed around the world, what happened?


1it [00:11, 11.52s/it]

...

Stage 1:../outputs/Financial_Markets_Course2_flanT5L/GQ.pickle has been saved


Token indices sequence length is longer than the specified maximum sequence length for this model (519 > 512). Running this sequence through the model will result in indexing errors


Stage 2:../outputs/Financial_Markets_Course2_flanT5L/QG+QA.pickle has been saved


100%|██████████| 51/51 [01:07<00:00,  1.32s/it]


Stage 3:../outputs/Financial_Markets_Course2_flanT5L/GQ+QA+GQ.pickle has been saved
Stage 3:../outputs/Financial_Markets_Course2_flanT5L/questions.txt has been saved
Stage 4:../outputs/Financial_Markets_Course2_flanT5L/questions.txt has been saved


In [2]:
import json
import copy
import warnings
import numpy as np 
import torch
# Disable all warning messages
warnings.filterwarnings("ignore")
import os
from tqdm.auto import tqdm
tqdm.pandas()
name = 'Financial_Markets_Course2_flanT5L'

dir_path = '../outputs/'+name+'/'
if not os.path.isdir(dir_path):
  os.mkdir(dir_path)

with open('/home/ubuntu/Questions_generation/Financial Markets Course 2.json') as f:
  result = json.load(f)
sections = [s['summary']['text'] if 'summary' in s.keys() and 'text' in s['summary'].keys() else None for k,s in result['sections'].items() ]
org_text = [s['original']['text'] if 'original' in s.keys() and 'text' in s['original'].keys() else None  for k,s in result['sections'].items() ]

min_words_in_section=60
sections = list(filter(lambda x: x is not None and len(x.split())>min_words_in_section, sections))
#org_text = list(filter(lambda x: x is not None and len(x.split())>min_words_in_section, org_text))
from MCQ import flanT5MCQ

generator_args = {
    "max_new_tokens":150,
#"max_length": 256,
"num_beams": 10, #20
"length_penalty":-0.5, #Since the score is the log likelihood of the sequence (i.e. negative), length_penalty > 0.0 promotes longer sequences, while length_penalty < 0.0 encourages shorter sequences.
"no_repeat_ngram_size": 3,
#'force_words_ids':[[58]],#token of `?` -cannot use constrained beam search with grouped beam search, while `diversity_penalty` can be used only with group beam search.
'top_p' :0.955,
#'do_sample':True,
'diversity_penalty':float(10), #note diversity is calculated between groups, the final scores are across all outputs, therfore the results with highest scores may be from one group and the diversity calcultion won't be effective for large groups
'num_beam_groups':10,#20 
"return_dict_in_generate" :True,
'output_scores':True,
"early_stopping": True, 
'num_return_sequences':8
}

answers_generator_args = {
    "max_new_tokens":150,
    #"max_length": 256,
    "num_beams": 8,#10
    "length_penalty":0.2,
    #"length_penalty": 1.5, #Since the score is the log likelihood of the sequence (i.e. negative), length_penalty > 0.0 promotes longer sequences, while length_penalty < 0.0 encourages shorter sequences.
    "no_repeat_ngram_size": 3,
    #'force_words_ids':[tokenizer.encode(['.'])],
    'top_p' :0.97,
    'diversity_penalty':float(8),
    'num_beam_groups':8,#10,
    "return_dict_in_generate" :True,
    'output_scores':True,
    "early_stopping": True,
    'num_return_sequences':5
}
import json
import copy
import warnings
import numpy as np 
import torch
# Disable all warning messages
warnings.filterwarnings("ignore")
import os
from tqdm.auto import tqdm
tqdm.pandas()
name = 'Financial_Markets_Course2_flanT5L'

dir_path = '../outputs/'+name+'/'
if not os.path.isdir(dir_path):
  os.mkdir(dir_path)

with open('/home/ubuntu/Questions_generation/Financial Markets Course 2.json') as f:
  result = json.load(f)
sections = [s['summary']['text'] if 'summary' in s.keys() and 'text' in s['summary'].keys() else None for k,s in result['sections'].items() ]
org_text = [s['original']['text'] if 'original' in s.keys() and 'text' in s['original'].keys() else None  for k,s in result['sections'].items() ]

min_words_in_section=60
sections = list(filter(lambda x: x is not None and len(x.split())>min_words_in_section, sections))
#org_text = list(filter(lambda x: x is not None and len(x.split())>min_words_in_section, org_text))
short_answers_generator_args = copy.deepcopy(answers_generator_args)
short_answers_generator_args["length_penalty"]=-0.6
mcq = flanT5MCQ(generator_args=generator_args,answers_generator_args=answers_generator_args,short_answers_generator_args=short_answers_generator_args)

In [7]:
import re
def is_similar(string, patterns):
    # compile the regex pattern to remove spaces and punctuation marks
    string = string.replace('_','')
    pattern = r'[^\w\s]'
    regex = re.compile(pattern)

    # remove spaces and punctuation marks from the input string and the patterns
    string = regex.sub('', string)
    patterns = [regex.sub('', p) for p in patterns]

    # check if the input string is exactly the same as one of the patterns
    return string.lower().strip() in patterns
answers_black_list =['we','they','the authors','authors','author','the author','you','you are','we are',
                                'the speaker','speaker','the lecturer','lecturer',
                                'he','he is','she','she is',
                                  'I','these','those','they are','we do','it','is is']
inputs = ['we','We','we...','They!','  Go','THEY  _ ']
[is_similar(ak,answers_black_list) for ak in inputs]

[True, True, True, True, False, False]

In [20]:
def text_slicer( text):
        print(len(mcq.tokenizer(text, return_tensors="pt")["input_ids"][0]))
        if len(mcq.tokenizer(text, return_tensors="pt")["input_ids"][0]) <= mcq.tokenizer.model_max_length :
            return [text]
        chunks = []
        chunk = []
        length = 0
        for sentence in nltk.tokenize.sent_tokenize(text):
            _len = len(mcq.tokenizer.tokenize(sentence))
            if length + _len <= mcq.tokenizer.model_max_length :
                length += _len
                chunk.append(sentence)
            elif not chunk:
                # Can a sentence be applicable for splitting on to chunks?
                chunks.append(sentence.strip())
                length = 0
            else:
                chunks.append(' '.join(chunk).strip())
                chunk = [sentence]
                length = _len
        if chunk:
            chunks.append(' '.join(chunk).strip())
        return chunks
sections_chunks = []
sections_n = []
for i,cur_section in enumerate(sections):
    cur_section_chunks = text_slicer(cur_section)
    n_chunks = len(cur_section_chunks)
    sections_chunks.extend(cur_section_chunks)
    if n_chunks==1:
        sections_n.append(float(i))
    else:
        sections_n.extend([i+0.1*k for k in range(n_chunks)])
print('='*100)
[text_slicer(cur_section) for cur_section in sections_chunks]

376
244
963
534
1167
664
376
244
512
452
473
62
510
485
174
489
176


[[" The crisis began with bubbles in the stock market, housing market, and also in the commodities market. It's a financial crisis that's bigger than any since the Great Depression of the 1930's. There's many different ways of thinking about a crisis like this. And I wanted to focus on one way that people think about it in terms of probability models. So, that's not the only way, it's not necessarily my favorite way. Excuse my cold. I didn't bring any water. I hope I make it through this lecture.  There was a pre-break around 2000 when the stock market collapsed around the world. But then they came back again after 2003 and they were on another boom, like a roller coaster ride. That's the narrative story. And then, what happened is, we see a bunch of institutional collapses. We saw bank failures in the U.S. and then, we saw international cooperation to prevent this from spreading like a disease. So, we had governments all over the world bailing out their banks and other companies. That

In [19]:
len(mcq.tokenizer(sections, return_tensors="pt")["input_ids"][0]) 

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`input_ids` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [12]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True